# Targeting high value customers based on customer demographics and attributes.
> This project was done under the umbrella of KPMG internship experience. I was provided data sets of an organization targeting a client who wants a feedback from us on their dataset quality and how this can be improved.

### Background
- Sprocket Central Pty Ltd, a medium size bikes & cycling accessories organisation
- needs help with its customer and transactions data
- how to analyse it to help optimise its marketing strategy effectively.

### Datasets
- New Customer List
- Customer Demographic
- Customer Addresses
- Transactions data in the past 3 months


### Task
- Exploratory Data Analysis to understand the data and its quality
- Model building to predict the high value customers
- Results and recommendations

In [58]:
# Importing libraries

import pandas as pd
import numpy as np
import datetime as dt

In [59]:
# Importing data
xls = pd.ExcelFile('KPMG_VI_New_raw_data_update_final.xlsx')

Transactions = pd.read_excel(xls, 'Transactions', skiprows=1)
NewCustomerList = pd.read_excel(xls, 'NewCustomerList', skiprows=1)
Demographic = pd.read_excel(xls, 'CustomerDemographic', skiprows=1)
Address = pd.read_excel(xls, 'CustomerAddress', skiprows=1)

### Checking correlation and common columns among the sheets

In [60]:
# Making variables to store the columns of each dataframe

transactions_columns = Transactions.columns
demographic_columns = Demographic.columns
newcustomerlist_columns = NewCustomerList.columns
address_columns = Address.columns

transactions_columns.name = 'transactions_columns'
demographic_columns.name = 'demographic_columns'
newcustomerlist_columns.name = 'newcustomerlist_columns'
address_columns.name = 'address_columns'

In [61]:
# A code I prompted to generate a dataframe to generate a sheet vs column presence table
# This shows if a column header is present across multiple dataframes or not

def generate_presence_dataframe(*columns):
    """
    Generate a DataFrame to show the presence of attributes in each column.

    Args:
        *columns: Variable number of pandas DataFrame columns.

    Returns:
        A pandas DataFrame with the presence of attributes in each column.
        The column headers are based on the names of the passed columns,
        or generic names if the columns don't have names.
        The displayed columns are in the same order of the passed column arguments.
        The DataFrame is sorted based on the number of '1' values horizontally (across the rows).

    """
    # Step 1: Convert the column(s) to set(s)
    column_sets = [set(col) for col in columns]

    # Step 2: Create a set of all unique attributes from the column(s)
    all_attributes = sorted(list(set().union(*column_sets)))

    # Step 3: Create a dictionary to store the presence of attributes in each column
    presence_dict = {'Attributes': all_attributes}
    for i, col in enumerate(columns):
        column_name = col.name if col.name else f'Column {i+1}'
        presence_dict[column_name] = [1 if attr in col else 0 for attr in all_attributes]

    # Step 4: Create a DataFrame from the presence dictionary
    presence_df = pd.DataFrame(presence_dict)

    # Step 5: Sort the dataframe based on the number of '1' values horizontally (across the rows)
    presence_df = presence_df.iloc[presence_df.iloc[:, 1:].sum(axis=1).sort_values(ascending=False).index]

    # Reset the index
    presence_df = presence_df.reset_index(drop=True)

    return presence_df


In [62]:
columns_presence_df = generate_presence_dataframe(transactions_columns, demographic_columns, address_columns,newcustomerlist_columns)
columns_presence_df

,Attributes,transactions_columns,demographic_columns,address_columns,newcustomerlist_columns
0,customer_id,1,1,1,0
1,DOB,0,1,0,1
2,first_name,0,1,0,1
3,tenure,0,1,0,1
4,state,0,0,1,1
5,property_valuation,0,0,1,1
6,postcode,0,0,1,1
7,past_3_years_bike_related_purchases,0,1,0,1
8,owns_car,0,1,0,1
9,job_title,0,1,0,1


### Highlights of the corelation analysis
We are supposed to combine the data from the three sheets (Customer Demographic, Customer Addresses, Transactions) and then make a Master Sheet which would be a training set where we would train our model to predict the high value customers.
- can use **customer_id** as the primary key to combine the data
- there are some irrelevant columns in plain sight which can be dropped
- there are rows with missing data which can be dropped as well, since it would affect the model building
- the DOB can be converted to age, and then we can perform an analysis with different age groups
- a **polynomial regression** model can be used to predict the high value customers
- values for the states, DOB and gender can be made uniform.

### Building a master Dataframe to train the model
- The master dataframe is built by combining the three sheets using the customer_id as the primary key
- The irrelevant columns are dropped
- reference for [pandas merge](https://www.youtube.com/watch?v=h4hOPGo4UVU)

In [63]:
# Merging the dataframes using customer_id as the key
merged_df = pd.merge(Demographic, Address, on='customer_id', how='outer')
master_df = pd.merge(merged_df, Transactions, on='customer_id', how='outer')
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20510 entries, 0 to 20509
Data columns (total 30 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   customer_id                          20510 non-null  int64         
 1   first_name                           20504 non-null  object        
 2   last_name                            19849 non-null  object        
 3   gender                               20504 non-null  object        
 4   past_3_years_bike_related_purchases  20504 non-null  float64       
 5   DOB                                  20047 non-null  object        
 6   job_title                            18027 non-null  object        
 7   job_industry_category                17180 non-null  object        
 8   wealth_segment                       20504 non-null  object        
 9   deceased_indicator                   20504 non-null  object        
 10  default   

In [64]:
# List of columns to drop
columns_to_drop = ['transaction_id', 'product_id', 'first_name', 'last_name', 'default', 'country','address', 'deceased_indicator']
# Create a new DataFrame by dropping the specified columns
master_stripped = master_df.drop(columns=columns_to_drop)
master_stripped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20510 entries, 0 to 20509
Data columns (total 22 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   customer_id                          20510 non-null  int64         
 1   gender                               20504 non-null  object        
 2   past_3_years_bike_related_purchases  20504 non-null  float64       
 3   DOB                                  20047 non-null  object        
 4   job_title                            18027 non-null  object        
 5   job_industry_category                17180 non-null  object        
 6   wealth_segment                       20504 non-null  object        
 7   owns_car                             20504 non-null  object        
 8   tenure                               20047 non-null  float64       
 9   postcode                             20478 non-null  float64       
 10  state     

### Making the dates uniform and adding the age column

In [65]:
# Making the DOB and transaction_date columns into datetime objects
master_stripped['DOB'] = pd.to_datetime(master_stripped['DOB'])
master_stripped['transaction_date'] = pd.to_datetime(master_stripped['transaction_date'])

In [66]:
import pandas as pd

# Assuming your DataFrame is called master_stripped

# Define the start and end dates of the dataset
start_date = pd.to_datetime('2017-01-01')
end_date = pd.to_datetime('2017-12-31')

# Convert "DOB" column to datetime format
master_stripped['DOB'] = pd.to_datetime(master_stripped['DOB'])

# Calculate customer age based on "DOB" column
master_stripped['customer_age'] = ((end_date - master_stripped['DOB']).dt.days // 365.25)


In [67]:
# Define the reference date
reference_date = pd.to_datetime('2018-01-01')

# Convert "product_first_sold_date" column to timedelta format
master_stripped['product_first_sold_date'] = pd.to_datetime(master_stripped['product_first_sold_date'], unit='D')

# # Calculate product age based on "product_first_sold_date" column
# master_stripped['product_age'] = (reference_date - pd.to_datetime(0)).days - master_stripped['product_first_sold_date'].dt.days
# master_stripped['product_age'] = (master_stripped['product_age'] / 365.25).abs().fillna(np.nan).round().astype('Int64')


In [68]:
master_stripped.head()

,customer_id,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,owns_car,tenure,postcode,...,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date,customer_age
0,1,F,93.0,1953-10-12,Executive Secretary,Health,Mass Customer,Yes,11.0,2016.0,...,0.0,Approved,OHM Cycles,Standard,medium,medium,235.63,125.07,2075-05-12,64.0
1,1,F,93.0,1953-10-12,Executive Secretary,Health,Mass Customer,Yes,11.0,2016.0,...,1.0,Approved,Solex,Standard,medium,medium,1577.53,826.51,2078-03-21,64.0
2,1,F,93.0,1953-10-12,Executive Secretary,Health,Mass Customer,Yes,11.0,2016.0,...,1.0,Approved,Trek Bicycles,Road,low,small,1720.70,1531.42,2073-07-22,64.0
3,1,F,93.0,1953-10-12,Executive Secretary,Health,Mass Customer,Yes,11.0,2016.0,...,0.0,Approved,Norco Bicycles,Standard,medium,medium,360.40,270.30,2073-09-10,64.0
4,1,F,93.0,1953-10-12,Executive Secretary,Health,Mass Customer,Yes,11.0,2016.0,...,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,2075-08-11,64.0


### Making the States and Genders uniform

In [69]:
# Replace 'New South Wales' with 'NSW' and Victoria with 'VIC'
master_stripped['state'] = master_stripped['state'].replace('New South Wales', 'NSW')
master_stripped['state'] = master_stripped['state'].replace('Victoria', 'VIC')

# Replace 'Female'and 'Femal' with 'F' and 'Male' with 'M'
master_stripped['gender'] = master_stripped['gender'].replace('Female', 'F')
master_stripped['gender'] = master_stripped['gender'].replace('Femal', 'F')
master_stripped['gender'] = master_stripped['gender'].replace('Male', 'M')

In [70]:
# Obtain the unique values of the "state" and the gender columns
states = master_stripped['state'].unique()
genders = master_stripped['gender'].unique()

In [71]:
print(states)
print(genders)

['NSW' nan 'QLD' 'VIC']
['F' 'M' 'U' nan]


### Converting the non Boolean values to Boolean values

In [72]:
# Fixing the owns_car column

# Step 1: Replace blank values with NaN
master_stripped['owns_car'].replace('', pd.NA, inplace=True)

# Step 2: Convert 'Yes' to True and 'No' to False
mapping = {'Yes': True, 'No': False}
master_stripped['owns_car'] = master_stripped['owns_car'].map(mapping)

# Step 3: Fill NaN values with False
master_stripped['owns_car'].fillna(False, inplace=True)

In [73]:
# Fixing the owns_car column

# Step 1: Replace blank values with NaN
master_stripped['order_status'].replace('', pd.NA, inplace=True)

# Step 2: Convert 'Yes' to True and 'No' to False
mapping = {'Approved': True, 'Cancelled': False}
master_stripped['order_status'] = master_stripped['order_status'].map(mapping)

# Step 3: Fill NaN values with False
master_stripped['order_status'].fillna(False, inplace=True)

In [74]:
# Fixing the online_order column
# Step 1: Replace blank values with NaN
master_stripped['online_order'].replace('', pd.NA, inplace=True)

# Step 2: Convert the column to boolean type
master_stripped['online_order'] = master_stripped['online_order'].astype('boolean')

### Fixing bad floats

In [75]:
master_stripped['tenure'] = master_stripped['tenure'].astype('Int64')
master_stripped['past_3_years_bike_related_purchases'] = master_stripped['past_3_years_bike_related_purchases'].astype('Int64')
master_stripped['postcode'] = master_stripped['postcode'].astype('Int64')
master_stripped['property_valuation'] = master_stripped['property_valuation'].astype('Int64')
master_stripped['customer_age'] = master_stripped['customer_age'].astype('Int64')

In [76]:
master_stripped.to_csv('master_stripped.csv', index=False)

### Analysing Null Values of the Master Dataframe

In [77]:
# Function to analyse the missing data

def analyze_null_values(dataframe):
    total_null_values = dataframe.isnull().sum()
    total_values = dataframe.count().sort_values(ascending=True)
    null_values_percentage = (total_null_values / total_values) * 100
    missing_data = pd.concat(
        {'Null Values': total_null_values, 'Total Values': total_values, 'Percentage of Missing Values': null_values_percentage},
        axis=1
    )
    
    missing_data = missing_data.sort_values(by='Percentage of Missing Values', ascending=False)
    
    return missing_data


In [78]:
analyze_null_values(master_stripped)

,Null Values,Total Values,Percentage of Missing Values
job_industry_category,3330,17180,19.383003
job_title,2483,18027,13.773784
online_order,870,19640,4.429735
brand,707,19803,3.570166
product_first_sold_date,707,19803,3.570166
standard_cost,707,19803,3.570166
product_size,707,19803,3.570166
product_class,707,19803,3.570166
product_line,707,19803,3.570166
list_price,510,20000,2.550000


In [81]:
# Making a datset with the dropped null values
master_null = master_stripped.dropna()
analyze_null_values(master_null)

,Null Values,Total Values,Percentage of Missing Values
customer_id,0,14062,0.0
transaction_date,0,14062,0.0
product_first_sold_date,0,14062,0.0
standard_cost,0,14062,0.0
list_price,0,14062,0.0
product_size,0,14062,0.0
product_class,0,14062,0.0
product_line,0,14062,0.0
brand,0,14062,0.0
order_status,0,14062,0.0


### Highlights of the Null Values Analysis
- job_title and job_industry_category have a lot of null values
- we can make two models, one where the job_title and job_industry_category are dropped, and one where they are not dropped
- learn about RFM analysis and KDD method
- followed a tutorial, the guy dropped all the null values no questions asked.